## Problem 31: Reconstruct a String from its k-mer Composition

#### String Reconstruction Problem
Reconstruct a string from its k-mer composition.

> Given: An integer k followed by a list of k-mers Patterns.

> Return: A string Text with k-mer composition equal to Patterns. (If multiple answers exist, you may return any one.)


<br>

``Plan / Putting things together:``

1) making a dre bogle graph. where edges are the kmers, and nodes are the prefixes/suffixes (problem 28)
    - we'll get an adj list like 
    
```
        ACC -> CCA
        
        CTT -> TTA
        
        GCT -> CTT
        
        GGC -> GCT
        
        TAC -> ACC
        
        TTA -> TAC
```


2) We'll apply the eulerian path alg to those nodes [problem 30]
    - we'll get a result like 
    ```GGC->GCT->CTT->TTA->TAC->ACC->CCA```


3) then we'll reconstruct the string from that sequence
    - we'll get 
    ```GGCTTACCA```

In [47]:
# ---- INPUT -----

text = """
4
CTTA
ACCA
TACC
GGCT
GCTT
TTAC
"""

In [48]:
stripped = [ln.strip() for ln in text.splitlines() if ln.strip()]
k = stripped[0]
patterns = stripped[1:]

patterns

['CTTA', 'ACCA', 'TACC', 'GGCT', 'GCTT', 'TTAC']

In [49]:
k1 = len(patterns[0]) - 1
patterns[:k1]

['CTTA', 'ACCA', 'TACC']

In [50]:
adj = {}
k = len(patterns[0])
k1 = k - 1

for p in patterns:
    pref = p[:k1]       # (k-1)-mer prefix
    suff = p[1:]        # (k-1)-mer suffix
    
    if pref not in adj:
        adj[pref] = []
    adj[pref].append(suff)   # we'll keep duplicates (multiple edges)
    
adj

{'CTT': ['TTA'],
 'ACC': ['CCA'],
 'TAC': ['ACC'],
 'GGC': ['GCT'],
 'GCT': ['CTT'],
 'TTA': ['TAC']}

In [51]:
def display_adjacency_list(adj_list):
    for u in sorted(adj):
        edge_list = ','.join(sorted(adj[u]))
        print(f"{u} -> {edge_list}")

display_adjacency_list(adj)

ACC -> CCA
CTT -> TTA
GCT -> CTT
GGC -> GCT
TAC -> ACC
TTA -> TAC


In [52]:
# Constructing and finding a eulerian path [Problem 30]

diff = {}  

for u, vs in adj.items():
    
    # add outdegrees
    diff[u] = diff.get(u, 0) + len(vs)      
    
    # subtract indegrees
    for v in vs:
        diff[v] = diff.get(v, 0) - 1    
        
#     print(diff)

    
start = next((n for n in diff if diff[n] == 1), None)
end = next((n for n in diff if diff[n] == -1), None)

print("\nstart, end =", (start,end))
diff


start, end = ('GGC', 'CCA')


{'CTT': 0, 'TTA': 0, 'ACC': 0, 'CCA': -1, 'TAC': 0, 'GGC': 1, 'GCT': 0}

In [53]:
# Constructing and finding a eulerian path [Problem 30]
def eulerian_cycle(adj, start):

    # making a shallow copy 
    g = {u: list(vs) for u, vs in adj.items()}


#   print("start: ", start)
    stack, cycle = [start], []

    while stack:
        v = stack[-1]
        
        # if v exists, we pop from the adj_list, and append to stack
        if g[v]:
            w = g[v].pop()   # consume edge v->w
            stack.append(w)
            
        # if no vs under that g (all popped for that g), then we pop from stack and add to cycle
        else:
            popped = stack.pop()
            cycle.append(popped)
            
#         print("\nstack: ", stack)
#         print("cycle: ", cycle)
#         print(g)

    # reverse to correct order
    cycle.reverse()  
    
    return cycle


In [54]:
def reconstruct_string_from_genome_path(patterns):
    
    reconstruction = patterns[0]

    for i in range(1, len(patterns)):
        reconstruction += patterns[i][-1]

    return reconstruction

In [55]:
# add an edge from start to end
adj[end] = adj.get(end, []) + [start]

# eul path: get eul cycle with the correct start, then remove the final edge we augmented
cycle = eulerian_cycle(adj, start)[:-1]

path_str = "->".join(cycle)
print(path_str)
cycle

GGC->GCT->CTT->TTA->TAC->ACC->CCA


['GGC', 'GCT', 'CTT', 'TTA', 'TAC', 'ACC', 'CCA']

In [56]:
# reconstruction algorithm from [Problem 25]
reconstructed = reconstruct_string_from_genome_path(cycle)
reconstructed

'GGCTTACCA'